In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from glob import glob

In [2]:
def open_cfg(file):
    with open(file, 'rt') as f:
        data = json.loads(f.read())

    for key in list(data.keys()):
        if type(data[key]) == list:
            data[key] = tuple(data[key])
    return data

def dicts_equal(dict1, dict2):
    keys1 = tuple(sorted(list(dict1.keys())))
    keys2 = tuple(sorted(list(dict2.keys())))
    if keys1 != keys2:
        return False
    for key in keys1:
        if dict1[key] != dict2[key]:
            return False
    return True

In [3]:
MODELS = ['ISIBrno_model', 'CNN_model', 'RNN_model', 'ECGConvEncoder']
TRAIN_DATASETS = [['ptb_xl'], ['ptb_xl', 'ningbo'], ['ptb_xl', 'ningbo', 'georgia']]
EXP2_DATASETS = ['sph', 'code15']

In [4]:
results = sorted(glob('results/*.csv'))
print(len(results))
results = {log: pd.read_csv(log) for log in results}
print(len(results))
results = {key.removeprefix('results/').removesuffix('.csv'): val for key, val in results.items()}
results = {key: {'log': val, 'cfg': open_cfg(f'results/{key}.cfg')} for key, val in results.items() if os.path.isfile(f'results/{key}.cfg')}
print(len(results))
results = {key: val for key, val in results.items() if val['cfg']['ecg_encoder_model'] in MODELS}
print(len(results))

98
98
94
94


In [18]:
for model in MODELS:
    print()
    print(model)
    model_results = {key: val for key, val in results.items() if val['cfg']['ecg_encoder_model'] == model}
    if model == 'ISIBrno_model':
        model_results = {key: val for key, val in model_results.items() if val['cfg']['device'] == 'cuda:0'}
    if model == 'ECGConvEncoder':
        model_results = {key: val for key, val in model_results.items() if val['cfg']['device'] == 'cuda:3'}
    print(len(model_results))

    for datasets in TRAIN_DATASETS:
        print()
        #print(datasets)
        ds_results = {key: val for key, val in model_results.items() if val['cfg']['train_datasets'] == tuple(datasets)}
        #print(len(ds_results))


        for train_ds in datasets:
            vals = [res['cfg']['test_metrics'][train_ds]['mean_rocaucs'] for res in ds_results.values()]
            print(f'{train_ds} test metrics: {np.mean(vals):.3f}+/-{np.std(vals):.3f}')

        
        for train_ds in datasets:
            vals = [res['cfg']['zero_shot_test_metrics'][train_ds]['mean_rocaucs'] for res in ds_results.values()]
            print(f'{train_ds} zero-shot metrics: {np.mean(vals):.3f}+/-{np.std(vals):.3f}') 


        for exp2_ds in EXP2_DATASETS:
            vals = [res['cfg']['exp2_metrics_trained'][exp2_ds]['mean_rocaucs'] for res in ds_results.values()]
            print(f'{exp2_ds} trained metrics: {np.mean(vals):.3f}+/-{np.std(vals):.3f}')


        for exp2_ds in EXP2_DATASETS:
            vals = [res['cfg']['exp2_metrics_untrained'][exp2_ds]['mean_rocaucs'] for res in ds_results.values()]
            print(f'{exp2_ds} zero-shot metrics: {np.mean(vals):.3f}+/-{np.std(vals):.3f}')
        #plt.figure(figsize=(30, 5))
        #for result in ds_results.values():
        #    plt.plot(result['log']['valid_mean_rocaucs'])
        #plt.grid()
        #plt.show()


ISIBrno_model
15

ptb_xl test metrics: 0.842+/-0.011
ptb_xl zero-shot metrics: 0.614+/-0.034
sph trained metrics: 0.848+/-0.034
code15 trained metrics: 0.870+/-0.035
sph zero-shot metrics: 0.744+/-0.043
code15 zero-shot metrics: 0.798+/-0.080

ptb_xl test metrics: 0.823+/-0.020
ningbo test metrics: 0.844+/-0.006
ptb_xl zero-shot metrics: 0.632+/-0.060
ningbo zero-shot metrics: 0.575+/-0.072
sph trained metrics: 0.884+/-0.032
code15 trained metrics: 0.883+/-0.031
sph zero-shot metrics: 0.729+/-0.034
code15 zero-shot metrics: 0.755+/-0.120

ptb_xl test metrics: 0.831+/-0.027
ningbo test metrics: 0.825+/-0.057
georgia test metrics: 0.780+/-0.023
ptb_xl zero-shot metrics: 0.620+/-0.033
ningbo zero-shot metrics: 0.590+/-0.045
georgia zero-shot metrics: 0.579+/-0.046
sph trained metrics: 0.904+/-0.035
code15 trained metrics: 0.930+/-0.013
sph zero-shot metrics: 0.688+/-0.033
code15 zero-shot metrics: 0.594+/-0.093

CNN_model
15

ptb_xl test metrics: 0.829+/-0.015
ptb_xl zero-shot metrics: 0